In [101]:
import pandas as pd

import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Spectral3
from bokeh.models.glyphs import HBar
from bokeh.transform import dodge
from bokeh.core.properties import value

from bokeh.io import output_notebook

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

output_notebook()

Loading BokehJS ...

In [2]:
df=pd.read_csv("./data/endorsements-2020.csv")

In [40]:
df.head()

,date,position,city,state,endorser,endorsee,endorser party,source,order,category,body,district,points
0,2017-07-28,representative,NaN,MD,David Trone,John Delaney,D,https://twitter.com/davidjtrone/status/8909859...,NaN,Representatives,NaN,6.0,3
1,2019-01-02,governor,NaN,NY,Andrew Cuomo,Joe Biden,D,https://www.cnn.com/2019/01/02/politics/cuomo-...,NaN,Governors,NaN,NaN,8
2,2019-01-03,senator,NaN,CA,Dianne Feinstein,Joe Biden,D,https://www.politico.com/story/2019/01/03/dian...,NaN,Senators,NaN,NaN,6
3,2019-01-08,senator,NaN,DE,Thomas R. Carper,Joe Biden,D,https://fox61.com/2019/01/08/will-he-or-wont-h...,NaN,Senators,NaN,NaN,6
4,2019-01-12,representative,NaN,TX,Joaquin Castro,Julian Castro,D,https://www.dallasnews.com/news/politics/2019/...,NaN,Representatives,NaN,20.0,3


In [111]:
table = pd.pivot_table(df, values='points',index=['endorsee'],columns=['state'], aggfunc=np.sum).fillna(0)
table["points"]=table.sum(axis = 1, skipna = True)
table['homestate']=["MN",
        "VT",
        "TX",
        "NJ",
        "MA",
        "WA",
        "DE",
        "MD",
        "CO",
        "TX",
        "CA",
        "NY",
        "CO",
        "NY",
        "IN",
        "MT"
]

minus_home = []
home = []
ratio =[]
for index,row in table.iterrows():
    homestate_score=row[row['homestate']]
    minus_home.append(row['points']-homestate_score)
    home.append(homestate_score)
    if homestate_score == 0:
        ratio.append(0)
    else:
        ratio.append((row['points']-homestate_score)/homestate_score)

table['minus_home'] = minus_home
table['home'] = home
table['ratio'] = ratio
table.sort_values(by='ratio', ascending=False, inplace=True)
table.head()

state,AL,AZ,CA,CO,CT,DC,DE,FL,GA,IA,IL,IN,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NJ,NM,NV,NY,OH,OR,PA,PR,SC,SD,TN,TX,VA,VI,VT,WA,WI,points,homestate,minus_home,home,ratio
endorsee,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Pete Buttigieg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,25.0,IN,22.0,3.0,7.333333
Joe Biden,6.0,0.0,10.0,0.0,13.0,2.0,25.0,6.0,3.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,3.0,11.0,5.0,3.0,21.0,0.0,0.0,5.0,1.0,12.0,3.0,0.0,0.0,0.0,0.0,147.0,DE,122.0,25.0,4.880000
Beto O'Rourke,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,10.0,TX,7.0,3.0,2.333333
Bernie Sanders,0.0,0.0,4.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,48.0,VT,28.0,20.0,1.400000
Elizabeth Warren,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,31.0,3.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,MA,31.0,31.0,1.000000


In [115]:
source = ColumnDataSource(table)

candidates = table.index.tolist()

p = figure(x_range=candidates, y_range=(0, table['points'].max()), plot_height=500, title="Candiate Endorsement Points (order by ratio of points from homestates)",
           toolbar_location=None, tools="")

p.vbar(x="endorsee", top='points', width=0.2, source=source,
       color="#008fd5", legend=value("Overall Points"))

p.vbar(x="endorsee", top='home', width=0.2, source=source,
       color="#ff7400", legend=value("Home State Endorsement Points"))

p.xaxis.major_label_orientation = "vertical"
p.yaxis.axis_label = "Endorsement Points"

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

show(p)